In [ ]:
# prompt: create a interface for the above code

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

class ChatbotInterface:
    def __init__(self, model_name="gpt2"):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.chat_history = ""
        self.predefined_responses = {
            "hello": "Hi! How can I assist you today?",
            "how are you": "I'm doing great, thank you for asking!",
            "bye": "Goodbye! Take care!",
            "weather": "I'm not sure about the current weather, but I hope it's nice where you are!",
            "name": "I'm your friendly chatbot, here to help with anything you need!"
        }
        print("Chatbot: Hello! I'm your AI chatbot. Type 'exit' to end the chat.")

    def generate_response(self, prompt):
        prompt_lower = prompt.lower()
        for keyword, response in self.predefined_responses.items():
            if keyword in prompt_lower:
                self.chat_history += f"User: {prompt}\nChatbot: {response}\n"
                return response

        self.chat_history += f"User: {prompt}\nChatbot:"
        inputs = self.tokenizer.encode(self.chat_history, return_tensors="pt")
        outputs = self.model.generate(
            inputs,
            max_length=500,
            pad_token_id=self.tokenizer.eos_token_id,
            num_return_sequences=1,
            do_sample=True,
            temperature=0.7
        )
        response = self.tokenizer.decode(outputs[:, inputs.shape[-1]:][0], skip_special_tokens=True)
        self.chat_history += response.strip() + "\n"
        return response.strip()

    def run(self):
        while True:
            user_input = input("You: ")
            if user_input.lower() == "exit":
                print("Chatbot: Goodbye! Have a great day!")
                break

            chatbot_response = self.generate_response(user_input)
            print(f"Chatbot: {chatbot_response}")

if __name__ == "__main__":
    chatbot = ChatbotInterface()
    chatbot.run()


Chatbot: Hello! I'm your AI chatbot. Type 'exit' to end the chat.
You: how are you
Chatbot: I'm doing great, thank you for asking!
You: bye
Chatbot: Goodbye! Take care!
You: weather
Chatbot: I'm not sure about the current weather, but I hope it's nice where you are!
You: name
Chatbot: I'm your friendly chatbot, here to help with anything you need!
You: exit
Chatbot: Goodbye! Have a great day!
